In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


During the backpropagation stage, the error is calculated and gradient values are determined. The gradients are sent back to hidden layers and the weights are updated accordingly. The process of gradient determination and sending it back to the next hidden layer is continued until the input layer is reached. The gradient becomes smaller and smaller as it reaches the bottom of the network. Therefore, the weights of the initial layers will either update very slowly or remains the same. In other words, the initial layers of the network won’t learn effectively. Hence, deep network training will not converge and accuracy will either starts to degrade or saturate at a particular value. Although vanishing gradient problem was addressed using the normalized initialization of weights, deeper network accuracy was still not increasing.

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import cv2
from google.colab.patches import cv2_imshow

dataset_path = os.listdir('/content/gdrive/My Drive/training_set')

pet_types = os.listdir('/content/gdrive/My Drive/training_set')
print (pet_types)  # What kinds of Pets in Folder

print("Types of Pets found:", len(pet_types))


['cats', 'dogs']
Types of Pets found: 2


In [ ]:

pets = []

for item in pet_types:
 # Get all the file names
    all_types = os.listdir('/content/gdrive/My Drive/training_set' + '/' +item)


 # Add them to the list
    for pet in all_types:
        pets.append((item, str('/content/gdrive/My Drive/training_set' + '/' +item) + '/' + pet))

print(len(pets))

8015


In [ ]:
    
# Build a dataframe        
pets_df = pd.DataFrame(data=pets, columns=['cat or dog', 'image'])
print(pets_df.head())


  cat or dog                                              image
0       cats  /content/gdrive/My Drive/training_set/cats/cat...
1       cats  /content/gdrive/My Drive/training_set/cats/cat...
2       cats  /content/gdrive/My Drive/training_set/cats/cat...
3       cats  /content/gdrive/My Drive/training_set/cats/cat...
4       cats  /content/gdrive/My Drive/training_set/cats/cat...


In [ ]:

# check how many samples for each category are present
print("Total number of Pets in the dataset: ", len(pets_df))

pets_count = pets_df['cat or dog'].value_counts()

print("Images in each category: ")
print(pets_count)



Total number of Pets in the dataset:  8015
Images in each category: 
cats    4010
dogs    4005
Name: cat or dog, dtype: int64


In [ ]:
#Resizing of each Image o 224*224*3 and Appending both Categories into one Array

path = '/content/gdrive/My Drive/training_set/'


im_size = 224

images = []
labels = []

for i in pet_types:
    data_path = path + str(i)  # entered in 1st folder and then 2nd folder

    filenames = [i for i in os.listdir(data_path)]

     # will get the names of all images
    for f in filenames:
        img = cv2.imread(data_path + '/' + f)  # reading that image as array

        # will get the image as an array
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)
#labels 
print(len(images)) 
print(len(labels))  
#cv2_imshow(images[7000])

8015
8015


In [ ]:
# Transform the image array to a numpy type
images = np.array(images)
print(images.shape) 
#cv2_imshow(images[4000])  

(8015, 224, 224, 3)


In [ ]:
# Encoding from Aplhabetical Labels to Numerics

from sklearn.preprocessing import OneHotEncoder

y=pets_df['cat or dog'].values

y=y.reshape(-1,1)
print(y)

# define one hot encoding
encoder = OneHotEncoder(sparse=False)

# transform data
onehot = encoder.fit_transform(y)

print(onehot)
onehot.shape

[['cats']
 ['cats']
 ['cats']
 ...
 ['dogs']
 ['dogs']
 ['dogs']]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


(8015, 2)

In [ ]:
'''from sklearn.preprocessing import OneHotEncoder
##from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
y=pets_df['cat or dog'].values
print(y)
##y_labelencoder = LabelEncoder ()
#y = y_labelencoder.fit_transform (y)

y=y.reshape(-1,1)
print(y)

columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [0])], remainder='passthrough')
#print(columnTransformer)
Y= columnTransformer.fit_transform(y)
print(Y)
Y.shape'''

"from sklearn.preprocessing import OneHotEncoder\n##from sklearn.preprocessing import LabelEncoder\nfrom sklearn.compose import ColumnTransformer\ny=pets_df['cat or dog'].values\nprint(y)\n##y_labelencoder = LabelEncoder ()\n#y = y_labelencoder.fit_transform (y)\n\ny=y.reshape(-1,1)\nprint(y)\n\ncolumnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [0])], remainder='passthrough')\n#print(columnTransformer)\nY= columnTransformer.fit_transform(y)\nprint(Y)\nY.shape"

In [ ]:
#Shuffle the Dataset and Split data into train and test sets

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


images, Y = shuffle(images, y, random_state=1)

train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.30, random_state=415)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)
#cv2_imshow(images[5])
train_y


(5610, 224, 224, 3)
(5610, 1)
(2405, 224, 224, 3)
(2405, 1)


array([['dogs'],
       ['dogs'],
       ['cats'],
       ...,
       ['cats'],
       ['dogs'],
       ['dogs']], dtype=object)

In [ ]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
#from keras.utils import plot_model

from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow

from keras.initializers import glorot_uniform
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D

In [ ]:

def identity_block(X, f, filters):

    # Retrieve Filters
    F1, F2, F3 = filters  # F1=64,F2=64,256
    
    X_shortcut = X
    
    # First  layer
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    
    # Second  layer
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Third  layer
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    X = BatchNormalization(axis = 3)(X)

    # Final step: Add shortcut value to F(X), and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [ ]:

def convolutional_block(X, f, filters, s = 2):

    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    # First layer 
    X = Conv2D(F1, (1, 1), strides = (s,s))(X) # 1,1 is filter size
    X = BatchNormalization(axis = 3)(X)  # normalization on channels
    X = Activation('relu')(X)

      
    # Second layer  (f,f)=3*3 filter by default
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)


    # Third layer
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    X = BatchNormalization(axis = 3)(X)


    ##### SHORTCUT PATH #### 
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid')(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut)

    # Final step: Add shortcut value here, and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [ ]:

def ResNet50(input_shape=(224, 224, 3), classes=2):
    """
    Implementation of the ResNet50 architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    """

    # Define the input with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input) #3,3 padding

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2))(X) 
    X = BatchNormalization(axis=3)(X) 
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], s=1)
   
    # below 3 lines are the conv layers from convolutional_block function defined above
    #X = Conv2D(F1, (1, 1), strides = (s,s))(X)
    #X = Conv2D(F2, kernel_size = (f, f), strides = (1,1), padding = 'same')(X)
    #X = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '2a')(X)
   
    X = identity_block(X, 3, [64, 64, 256]) 
    #X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    #X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same')(X)
    #X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
  
    X = identity_block(X, 3, [64, 64, 256])
    #X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    #X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same')(X)
    #X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)


    # Stage 3 
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], s = 2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])

    # Stage 4 
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    # Stage 5 
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    # AVGPOOL 
    X = AveragePooling2D((2,2), name="avg_pool")(X)

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model


In [ ]:
model = ResNet50(input_shape = (224, 224, 3), classes = 2)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 64) 256         conv2d[0][0]                     
___________________________________________________________________________________________

In [ ]:
model.fit(train_x, train_y, epochs =30 , batch_size = 32) 

Epoch 1/30


ValueError: ignored

In [ ]:
preds = model.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
from matplotlib.pyplot import imread
img_path = '/content/dog.4003.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
print('Input image shape:', x.shape)
my_image = imread(img_path)
imshow(my_image)
print(model.predict(x))